# Assignment 2 - part-of-speech prediction from limited context

In this assignment, you will train classifiers that attempt, within a window of five words, to make a binary prediction about whether the third word belongs to a given part of speech (noun, verb, adjective, adverb), but using very limited information -- that is, the last two letters of the first, second, fourth, and fifth word of the sequence, and no information whatsoever directly from the third word itself.  You will strip out all punctuation (using the NLTK `WordPunctTokenizer`), lowercase, and remove stop words (using the NLTK English stop words list).

In other words, you will predict over samples that have two classes, P and not-P, where P is the selected part of speech to classify.  For example, from the sentence, "The quick brown fox jumped over the lazy dog.", we can select the following 5-word windows without stop words, "brown fox jumped lazy dog" and "quick brown fox jumped lazy".  If we select verbs as the part-of-speech we are classifying over, we get the instances <(wn,ox,zy,og),1>, since "jumped" is a verb, but <(ck,wn,ed,zy),0> because "fox" in that context is not.

This means that you will need to take into account the position of the last-two-letter feature:  "zy" as the fourth word's last two letters is different from "zy" as the fifth word's last two letters.  They are two features, say, `zy_4` and `zy_5`.

This will likely not actually work.  But it might!

You will create training and testing samples according to this procedure, and you will build a data structure that can be fed to a support vector machine (SVM) classifier.  You will train the classifier on the training data and evaluate it on the testing data. 

The work will be done in a .py module file in the same folder as this notebook.  **No modifications to this notebook will be graded.** We will run your module using this notebook or one we modify that you won't see in order to test your code.

The file you must create and add to the github repo is `mycode.py`, which will be imported here.  You can create your own notebooks or scripts to test it.  You can put any number of your own helper functions and also put optional parameters on any of the python functions mentioned here. You should also create a Markdown file, `notes.md`, to keep any **concise** notes and remarks about the assignment.  The code must run on mltgpu.

**This assignment is due Monday, 2022 March 7, at 23:59. There are 33 points and 5 bonus points.**

In [1]:
import mycode as mc

## Part 0 - preparation (2 points)

Fork this repository and create and add `mycode.py` and `notes.md`. 

## Part 1 - obtaining the text (3 points)

You will randomly select the given number of lines from the gzipped file we give you (so you will have to figure out how to access gzipped text files).  Explain how you implemented the random selection in `notes.md`. When we run it, it should give a new sample every time.

In [3]:
sampled_lines = mc.sample_lines("/scratch/UN-english.txt.gz", lines=100000)
sampled_lines[40000:40010]

['Following up on its participation in the World Conference Against Racism, the University has initiated "Ubuntu -- Awakening the Spirit of Humanity", a project involving intercultural exchange and dialogue leading to community-level educational and other values-based action.\n',
 'For further information, please contact the World Bank New York Office (tel. 1 (212) 355-5112).\n',
 '17. Introducing document HR/MEX/SEM.1/2002/BP.7, Ms. Gilda Pacheco noted that, in accordance with the provisions on training and education adopted by the Conference against Racism, there are four major types of challenges for the human rights movement: those related to efforts to ensure better knowledge and use of existing international norms and mechanisms for combating racism and discrimination; those that facilitate the harmonization of national legislation and public policies with international standards, with the participation of directly affected populations; those aimed at strengthening the skills of 

## Part 2 - creating the samples (7 points)

From the sampled lines, you will then randomly create the five-word samples.

You will tokenize the sentences and apply POS-tagging to them -- you need to do this before you create the samples, since POS-tagging needs context. You will then remove stop words and punctuation and lowercase the remainder.  Next, you will randomly, over the entire set of sentences, choose samples of five words in sequence, up to a certain limit.  You find the last two characters of the first, second, fourth, and fifth words, and create the type of structure specified up in the introduction to this assignment for each sample. The exact representation is up to you.

In [6]:
processed_sentences = mc.process_sentences(sampled_lines)
processed_sentences[40000:40010]

[[('following', 'VBG'),
  ('participation', 'NN'),
  ('world', 'NN'),
  ('conference', 'NN'),
  ('racism', 'NN'),
  ('university', 'NN'),
  ('initiated', 'VBN'),
  ('``', '``'),
  ('ubuntu', 'JJ'),
  ('--', ':'),
  ('awakening', 'VBG'),
  ('spirit', 'NN'),
  ('humanity', 'NN'),
  ("''", "''"),
  ('project', 'NN'),
  ('involving', 'VBG'),
  ('intercultural', 'JJ'),
  ('exchange', 'NN'),
  ('dialogue', 'NN'),
  ('leading', 'VBG'),
  ('community-level', 'JJ'),
  ('educational', 'JJ'),
  ('values-based', 'JJ'),
  ('action', 'NN')],
 [('information', 'NN'),
  ('please', 'NN'),
  ('contact', 'VB'),
  ('world', 'NN'),
  ('bank', 'NN'),
  ('new', 'JJ'),
  ('york', 'NN'),
  ('office', 'NN'),
  ('tel', 'NN'),
  ('1', 'CD'),
  ('212', 'CD'),
  ('355-5112', 'CD')],
 [('17.', 'CD'),
  ('introducing', 'VBG'),
  ('document', 'NN'),
  ('hr/mex/sem.1/2002/bp.7', 'NN'),
  ('ms.', 'FW'),
  ('gilda', 'NN'),
  ('pacheco', 'NN'),
  ('noted', 'VBD'),
  ('accordance', 'NN'),
  ('provisions', 'NNS'),
  ('train

In [7]:
all_samples = mc.create_samples(processed_sentences, samples=50000)

print(all_samples[25000:25010])

[('JJ', ['ed_0', 'sk_1', 'at_3', 'rt_4']), ('NN', ['sk_0', 'ar_1', 'rt_3', 'ns_4']), ('JJ', ['le_0', '16_1', 're_3', 'de_4']), ('NN', ['16_0', 'al_1', 'de_3', 'es_4']), ('NN', ['al_0', 're_1', 'es_3', 'nt_4']), ('VB', ['re_0', 'de_1', 'nt_3', 'gs_4']), ('NN', ['de_0', 'es_1', 'gs_3', 'ow_4']), ('NN', ['es_0', 'nt_1', 'ow_3', 'ge_4']), ('VB', ['nt_0', 'gs_1', 'ge_3', 'ed_4']), ('NN', ['gs_0', 'ow_1', 'ed_3', 'rt_4'])]


## Part 3 - convert the samples into a Pandas DataFrame (10 points)

Here, you will take the samples and create a table whose columns are the features and the class and whose rows are the samples.  All the features and the class will be binary.  Note that there may be many columns, in the hundreds or thousands depending on the diversity of the final two consonants of the non-stop-words in the dataset, but the sum of all rows will be five.

In [13]:
fulldf = mc.create_df(all_samples)
fulldf[25000:25010]

VB  68_4  ea_4  ap_0  rk_3  r._4  ür_1  te_0  4_3  ez_3  \
15_0 ng_1 en_3 ed_4   0     0     0     0     0     0     0     0    0     0   
ng_0 rk_1 ed_3 15_4   0     0     0     0     0     0     0     0    0     0   
rk_0 en_1 15_3 18_4   1     0     0     0     0     0     0     0    0     0   
15_0 18_1 ld_3 ur_4   0     0     0     0     0     0     0     0    0     0   
18_0 pe_1 ur_3 ee_4   0     0     0     0     0     0     0     0    0     0   
pe_0 ld_1 ee_3 ed_4   0     0     0     0     0     0     0     0    0     0   
al_0 ic_1 ts_3 nt_4   0     0     0     0     0     0     0     0    0     0   
     ts_1 ed_3 rs_4   0     0     0     0     0     0     0     0    0     0   
ts_0 nt_1 rs_3 al_4   0     0     0     0     0     0     0     0    0     0   
nt_0 ed_1 al_3 us_4   0     0     0     0     0     0     0     0    0     0   

                     ...  dh_1  tá_4  na_3  07_4  ue_0  tc_3  mr_3  .g_4  \
15_0 ng_1 en_3 ed_4  ...     0     0     0     0     0     0     0     0   
ng_0 rk_1 ed_3 15_4  ...     0     0     0     0     0     0     0     0   
rk_0 en_1 15_3 18_4  ...     0     0     0     0     0     0     0     0   
15_0 18_1 ld_3 ur_4  ...     0     0     0     0     0     0     0     0   
18_0 pe_1 ur_3 ee_4  ...     0     0     0     0     0     0     0     0   
pe_0 ld_1 ee_3 ed_4  ...     0     0     0     0     0     0     0     0   
al_0 ic_1 ts_3 nt_4  ...     0     0     0     0     0     0     0     0   
     ts_1 ed_3 rs_4  ...     0     0     0     0     0     0     0     0   
ts_0 nt_1 rs_3 al_4  ...     0     0     0     0     0     0     0     0   
nt_0 ed_1 al_3 us_4  ...     0     0     0     0     0     0     0     0   

                     gy_0  w-_3  
15_0 ng_1 en_3 ed_4     0     0  
ng_0 rk_1 ed_3 15_4     0     0  
rk_0 en_1 15_3 18_4     0     0  
15_0 18_1 ld_3 ur_4     0     0  
18_0 pe_1 ur_3 ee_4     0     0  
pe_0 ld_1 ee_3 ed_4     0     0  
al_0 ic_1 ts_3 nt_4     0     0  
     ts_1 ed_3 rs_4     0     0  
ts_0 nt_1 rs_3 al_4     0     0  
nt_0 ed_1 al_3 us_4     0     0  

[10 rows x 2347 columns]

## Part 4 - extract training and testing sets (3 points)

Here, you will create the training and testing datasets in order to train the model.  This will be based on a test percentage.  Round up if the percentage does not divide evenly into the sample size.  You will need to separate the class column into the y-values for the classifier.

In [14]:
train_X, train_y, test_X, test_y = mc.split_samples(fulldf, test_percent=20)
len(train_X), len(train_y), len(test_X), len(test_y)

(38553, 38553, 9639, 9639)

## Part 5 - train models (3 points)

You will then train and return two support vector machine (SVM) models using the sklearn SVC class.  You should allow a choice between linear and radial basis function kernels.

In [15]:
model_linear = mc.train(train_X, train_y, kernel='linear')
model_rbf = mc.train(train_X, train_y, kernel="rbf")
model_linear, model_rbf

(SVC(C=1, decision_function_shape='ovo', kernel='linear'),
 SVC(C=1, decision_function_shape='ovo', gamma=1))

## Part 6 - evaluate the models (5 points)

You will calculate and print precision, recall, and F-measure for the models on the test data. In `notes.md`, write down your comparison of these simple measures on the two models and any thoughts you might have on what they mean. (It could be very short, and since the samples do not stay stable between runs, you can save the evaluation scores in `notes.md` too.)

In [18]:
mc.eval_model(model_linear, test_X, test_y)
mc.eval_model(model_rbf, test_X, test_y)

              precision    recall  f1-score   support

           0       0.82      0.99      0.90      7838
           1       0.48      0.04      0.07      1801

    accuracy                           0.81      9639
   macro avg       0.65      0.51      0.48      9639
weighted avg       0.75      0.81      0.74      9639

              precision    recall  f1-score   support

           0       0.81      1.00      0.90      7838
           1       0.71      0.00      0.01      1801

    accuracy                           0.81      9639
   macro avg       0.76      0.50      0.45      9639
weighted avg       0.79      0.81      0.73      9639



## Part Bonus - try another sort of model from sklearn (5 points)

Write a separate, command-line script (not a notebook) uses `mycode.py` to do all of the above, except that it trains a non-SVM classifier model.  Any non-trivial model available in sklearn will do. Explain how to run your code and the results of your own evaluation in `notes.md`, including any observations or opinions you may have on the classifier method you used in comparison to SVM.

## Submission

Push to your fork of the GitHub repository (which must be made public) and submit the URL of your repository in Canvas.  You can submit this notebook with the output from your run, as long as you do not modify the code or text in it without permission from us.  